## <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and Google Colab</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Sunday, October 20th submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is about giving exposure about Tensorflow, its usage, Cloud services and help us in understanding the time taken to run computation on CPU and GPU. 

In this Project, we will work with CIFAR10 image dataset. 
The starter code to download the dataste using keras is given below. 
You should run this project on Google Colab. You would be using CPU, GPU.
Use tensorflow version 2.0. 

# In every line of code, please write a comment to briefly explain what that line is doing.
Your grades will be based on your understanding of the code you write! 

Note: The code you write should be your own!

# Task 1
Convert the features in a form that can be given as input to tensorflow library/functions

In this task you will perform data augmentation. That is, pre-process the data to make the model more robust. Most common data augmentation techniques are rotation, flips and histogram equalization. 
You can choose an augmentation technique of your choice. 

In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [7]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

datagen = ImageDataGenerator(width_shift_range=0.1, horizontal_flip=True)
datagen.fit(X_train)

# Task 2
Try to build a Neural Network model, train on the features and report the accuracy.
Report your observations on the time taken on GPU and TPUs. 

### 1. Create a CNN based model with 5 hidden layers and 100 hidden units each layer.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import time

startcnn = time.time()

batch_size = 32
hidden_layers = 100
epochs = 100
data_augmentation = False
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, hidden_layers)
y_test = keras.utils.to_categorical(y_test, hidden_layers)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(hidden_layers))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

endcnn = time.time()
print(startcnn-endcnn)

#### CNN - Time taken on GPU

#### CNN - Time taken on TPU

### 2. Create an LSTM based model with 2 hidden layers and 1024 hidden units in each layer.

In [ ]:
import numpy as np
from keras.layers import Dense, Embedding
from keras.layers import LSTM

x_train_flattened = np.asarray(x_train.flatten()).reshape(50000,1,3072)
x_test_flatttened = np.asarray(x_test.flatten()).reshape(10000,1,3072)

lstm_model = keras.models.Sequential()

# Add first LSTM layer with 1024 hidden units
lstm_model.add(keras.layers.LSTM(1024, input_shape=(1, 3072), dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

# Add second LSTM layer with 1024 hidden units
lstm_model.add(keras.layers.LSTM(1024, dropout=0.2, recurrent_dropout=0.2))

# Create sigmoid activation layer
lstm_model.add(keras.layers.Dense(10, activation='sigmoid'))

# Not sure if we need a softmax layer?

# Try different optimizers and optimizer configs
lstm_model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
lstm_model.summary()

In [ ]:
lstm_model.fit(x_train_flattened, y_train,
         batch_size=10,
         epochs=1,
         validation_data=(x_test_flattened, y_test))

#### LSTM - Time Taken on GPU

#### LSTM - Time Taken on TPU

# Task 3 (Extra credit, 25 points)
Run the above on a TPU and report the time taken to fit the models. 